In [3]:
import torch
import scvi
import torch.nn as nn
import torch.utils.data as data
import matplotlib.pyplot as plt
import anndata as ad
import scanpy as sc
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, precision_score, log_loss, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

Global seed set to 0
pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.cuda.set_device(1)
print("Device", device)
adata = ad.read_h5ad('/home/brunopsz/Data/GSE155249_COUNTS_NOT_NORMALIZED.h5ad')
adata

Device cuda


AnnData object with n_obs × n_vars = 77146 × 20692
    obs: 'barcode_name', 'Sample', 'Cluster'
    var: 'gene_name'
    layers: 'counts'

In [4]:
sc.pp.highly_variable_genes(
    adata,
    flavor="seurat_v3",
    n_top_genes=5000, #spróbować na więcej może 5000
    layer="counts",
    batch_key="Sample",
    subset=True,
)

scvi.model.SCANVI.setup_anndata(
    adata,
    batch_key = 'Sample',
    layer='counts',
    labels_key='Cluster',
    unlabeled_category='None'
)

In [5]:
scanvi = scvi.model.SCANVI(adata, n_latent = 10)
scanvi.train()
scanvi

INFO     Training for 104 epochs.                                                                                  


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Epoch 104/104: 100%|██████████| 104/104 [22:18<00:00, 13.46s/it, loss=1.51e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=104` reached.


Epoch 104/104: 100%|██████████| 104/104 [22:18<00:00, 12.87s/it, loss=1.51e+03, v_num=1]


ScanVI Model with the following params: 
unlabeled_category: None, n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: zinb
Training status: Trained

In [6]:
adata.obsm["X_scANVI"] = scanvi.get_latent_representation(adata)

In [7]:
adata.write_h5ad('/home/brunopsz/Data/scanvi_trained_adata_on_5000_genes_latent_10.h5ad')

In [8]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(adata.obs['Cluster'].to_numpy().reshape(-1,1))
print(ohe.categories_)

print('\n')

labels_one_hot = ohe.transform(adata.obs['Cluster'].to_numpy().reshape(-1,1))
print(labels_one_hot)

[array(['AT2, AT1 cells', 'B cells', 'CD4 CM T cells',
       'CD4 cytotoxic T cells', 'CD4 prolif. T cells',
       'CD8 cytotoxic T cells', 'CD8 cytotoxic TRM T cells',
       'CD8 prolif. T cells', 'Ciliated cells', 'Club, Basal cells',
       'DC1', 'DC2', 'Infected AT2, AT1 cells', 'Ionocytes', 'Mast cells',
       'Migratory DC', 'Mixed myeloid', 'MoAM1', 'MoAM2', 'MoAM3',
       'MoAM4', 'Plasma cells', 'Prolif. AM', 'TRAM1', 'TRAM2', 'Treg',
       'iNKT cells', 'pDC'], dtype=object)]


[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
class SimpleRNAseqClassifier(nn.Module):

    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        # Initialize the modules we need to build the network
        self.linear1 = nn.Linear(num_inputs, num_hidden)
        self.act_fn = nn.ReLU()
        self.linear2 = nn.Linear(num_hidden, num_outputs)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        return x
    
class scRNAseqDataset(data.Dataset):

    def __init__(self, data, labels):
        super().__init__()
        self.data = data
        self.label = labels

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        data_point = self.data[idx]
        data_label = self.label[idx]
        return data_point, data_label

In [17]:
cell_types = adata.obs['Cluster'].unique()
number_of_cell_types = len(cell_types)
print("Number of cell types = " + str(number_of_cell_types))

Number of cell types = 28


In [11]:
cell_type_counts = adata.obs.groupby(['Cluster'])['Cluster'].count()
cell_type_counts
class_weights = [(1-(count/sum(cell_type_counts))) for count in cell_type_counts]
class_weights = torch.FloatTensor(class_weights)
class_weights.to(device)

tensor([0.9966, 0.9935, 0.9281, 0.9758, 0.9870, 0.9414, 0.9576, 0.9848, 0.9810,
        0.9949, 0.9895, 0.9939, 0.9990, 0.9993, 0.9996, 0.9892, 0.9825, 0.9445,
        0.8438, 0.8391, 0.8509, 0.9254, 0.9984, 0.9736, 0.9727, 0.9894, 0.9872,
        0.9812], device='cuda:1')

In [16]:
accuracy_stats = {
    'train': [],
    'val': []
}

loss_stats = {
    'train': [],
    'val': []
}

f1_scores = {
    'train': [],
    'val': []
}

precision = {
    'train': [],
    'val': []
}

In [13]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    _, actual = torch.max(y_test.data, 1)

    correct_pred = (y_pred_tags == actual).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

def multi_f1(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    _, actual = torch.max(y_test.data, 1)
    return f1_score(actual.cpu(), y_pred_tags.cpu(), average='weighted', labels=np.unique(y_pred_tags.cpu()))


def multi_precision(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    _, actual = torch.max(y_test.data, 1)
    return precision_score(actual.cpu(), y_pred_tags.cpu(), average='weighted', labels=np.unique(y_pred_tags.cpu()))

In [14]:
LEARNING_RATE = 0.007

loss_module = nn.CrossEntropyLoss(weight = class_weights)

loss_module.to(device)

SimpleRNAseqClassifier(
  (linear1): Linear(in_features=10, out_features=64, bias=True)
  (act_fn): ReLU()
  (linear2): Linear(in_features=64, out_features=28, bias=True)
)

In [15]:
def index_to_cell_type(index):
    arr = np.zeros(number_of_cell_types)
    arr[index] = 1.0
    arr = arr.reshape(1,-1)
    return ohe.inverse_transform(arr)[0][0]

In [19]:
EPOCHS = 50
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=77)
cv_test_accuracy = []
cv_test_precision = []
cv_test_f1 = []
cv_test_roc = []

for i, (train_index, test_index) in enumerate(skf.split(adata.X, adata.obs['Cluster'])):
    
    model = SimpleRNAseqClassifier(num_inputs=adata.obsm["X_scANVI"].shape[1], num_hidden=64, num_outputs=number_of_cell_types)
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
    model.to(device)

    print(f'Begin training split {i}')
    train_dataset = scRNAseqDataset(torch.from_numpy(adata.obsm["X_scANVI"][train_index]), torch.from_numpy(labels_one_hot[train_index]))
    test_dataset = scRNAseqDataset(torch.from_numpy(adata.obsm["X_scANVI"][test_index]), torch.from_numpy(labels_one_hot[test_index]))

    train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    for epoch in tqdm(range(1, EPOCHS+1)):

        #Create writer to use TensorBoard
        writer = SummaryWriter('runs/model_CV_on_scanvi_5000_10_model')
        model_plotted = False
        
        # TRAINING
        train_epoch_loss = 0
        train_epoch_acc = 0
        train_epoch_f1 = 0
        train_epoch_precision = 0
        model.train()

        for X_train_batch, y_train_batch in train_loader:
            #Push data to GPU
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)

            if not model_plotted:
                writer.add_graph(model, X_train_batch)
                model_plotted = True
            
            optimizer.zero_grad()
            
            y_train_pred = model(X_train_batch)

            #Calculate train metrics
            train_loss = loss_module(y_train_pred, y_train_batch)
            train_acc = multi_acc(y_train_pred, y_train_batch)
            train_f1 = multi_f1(y_train_pred, y_train_batch)
            train_precision = multi_precision(y_train_pred, y_train_batch)

            train_loss.backward()
            optimizer.step()
            
            train_epoch_loss += train_loss.item()
            train_epoch_acc += train_acc.item()
            train_epoch_f1 += train_f1.item()
            train_epoch_precision += train_precision.item()
            
            
        # VALIDATION    
        with torch.no_grad():
            
            loss_stats['train'].append(train_epoch_loss/len(train_loader))
            writer.add_scalar('Loss/training', train_epoch_loss/len(train_loader), global_step = epoch + 1)
            
            f1_scores['train'].append(train_epoch_f1/len(train_loader))
            writer.add_scalar('F1_Score/training', train_epoch_f1/len(train_loader), global_step = epoch + 1)

            precision['train'].append(train_epoch_precision/len(train_loader))
            writer.add_scalar('Precision/training', train_epoch_precision/len(train_loader), global_step = epoch + 1)

            accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
            writer.add_scalar('Accuracy/training', train_epoch_acc/len(train_loader), global_step = epoch + 1)
            
        print(f'({i}) Epoch {epoch+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}')
    
    y_true_cells = []
    y_pred_cells = []

    y_true_prob = []
    y_pred_prob = []

    softmax = nn.Softmax(dim=1)
    
    for X_test_batch, y_test_batch in test_loader:
        #Push data to GPU
        X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)	

        y_true_prob.extend(y_test_batch)

        _, actual = torch.max(y_test_batch.data, 1)	

        actural_cell_types = [index_to_cell_type(index) for index in actual.cpu().numpy()]

        y_true_cells.extend(actural_cell_types)	

        outputs = model(X_test_batch)	

        y_pred_prob.extend(softmax(outputs))

        #Czy tu nie powininem użyć softmax przed max???
        _, predicted = torch.max(softmax(outputs), 1)	

        predicted_cell_types = [index_to_cell_type(index) for index in predicted.cpu().numpy()]

        y_pred_cells.extend(predicted_cell_types)

        
    y_true_prob = [y.cpu().numpy() for y in y_true_prob]
    y_pred_prob = [y.cpu().detach().numpy() for y in y_pred_prob]
    acc = accuracy_score(y_true_cells, y_pred_cells)

    print(f"({i} test accuracy = {acc})")
    cv_test_accuracy.append(acc)
    cv_test_precision.append(precision_score(y_true_cells,y_pred_cells, average='weighted'))
    cv_test_f1.append(f1_score(y_true_cells, y_pred_cells, average='weighted'))
    cv_test_roc.append(roc_auc_score(y_true_prob, y_pred_prob, average="weighted", multi_class="ovr"))

writer.close()

Begin training split 0


  0%|          | 0/50 [00:00<?, ?it/s]

(0) Epoch 001: | Train Loss: 0.91093 | Train Acc: 76.572
(0) Epoch 002: | Train Loss: 0.39983 | Train Acc: 88.894
(0) Epoch 003: | Train Loss: 0.32675 | Train Acc: 90.473
(0) Epoch 004: | Train Loss: 0.29465 | Train Acc: 91.149
(0) Epoch 005: | Train Loss: 0.27671 | Train Acc: 91.535
(0) Epoch 006: | Train Loss: 0.26502 | Train Acc: 91.806
(0) Epoch 007: | Train Loss: 0.25643 | Train Acc: 91.981
(0) Epoch 008: | Train Loss: 0.24954 | Train Acc: 92.209
(0) Epoch 009: | Train Loss: 0.24439 | Train Acc: 92.319
(0) Epoch 010: | Train Loss: 0.23955 | Train Acc: 92.442
(0) Epoch 011: | Train Loss: 0.23565 | Train Acc: 92.542
(0) Epoch 012: | Train Loss: 0.23187 | Train Acc: 92.631
(0) Epoch 013: | Train Loss: 0.22880 | Train Acc: 92.692
(0) Epoch 014: | Train Loss: 0.22594 | Train Acc: 92.770
(0) Epoch 015: | Train Loss: 0.22337 | Train Acc: 92.844
(0) Epoch 016: | Train Loss: 0.22074 | Train Acc: 92.963
(0) Epoch 017: | Train Loss: 0.21853 | Train Acc: 93.049
(0) Epoch 018: | Train Loss: 0.

  0%|          | 0/50 [00:00<?, ?it/s]

(1) Epoch 001: | Train Loss: 0.90872 | Train Acc: 76.583
(1) Epoch 002: | Train Loss: 0.39242 | Train Acc: 89.191
(1) Epoch 003: | Train Loss: 0.32192 | Train Acc: 90.774
(1) Epoch 004: | Train Loss: 0.29086 | Train Acc: 91.349
(1) Epoch 005: | Train Loss: 0.27361 | Train Acc: 91.703
(1) Epoch 006: | Train Loss: 0.26194 | Train Acc: 92.017
(1) Epoch 007: | Train Loss: 0.25360 | Train Acc: 92.182
(1) Epoch 008: | Train Loss: 0.24697 | Train Acc: 92.329
(1) Epoch 009: | Train Loss: 0.24131 | Train Acc: 92.443
(1) Epoch 010: | Train Loss: 0.23673 | Train Acc: 92.546
(1) Epoch 011: | Train Loss: 0.23275 | Train Acc: 92.694
(1) Epoch 012: | Train Loss: 0.22913 | Train Acc: 92.742
(1) Epoch 013: | Train Loss: 0.22579 | Train Acc: 92.880
(1) Epoch 014: | Train Loss: 0.22313 | Train Acc: 92.907
(1) Epoch 015: | Train Loss: 0.22040 | Train Acc: 92.985
(1) Epoch 016: | Train Loss: 0.21819 | Train Acc: 93.076
(1) Epoch 017: | Train Loss: 0.21613 | Train Acc: 93.123
(1) Epoch 018: | Train Loss: 0.

...

(7) Epoch 001: | Train Loss: 0.92996 | Train Acc: 74.886
(7) Epoch 002: | Train Loss: 0.40646 | Train Acc: 88.553
(7) Epoch 003: | Train Loss: 0.32956 | Train Acc: 90.241
(7) Epoch 004: | Train Loss: 0.29588 | Train Acc: 91.002
(7) Epoch 005: | Train Loss: 0.27696 | Train Acc: 91.448
(7) Epoch 006: | Train Loss: 0.26493 | Train Acc: 91.822
(7) Epoch 007: | Train Loss: 0.25641 | Train Acc: 91.982
(7) Epoch 008: | Train Loss: 0.24975 | Train Acc: 92.199
(7) Epoch 009: | Train Loss: 0.24443 | Train Acc: 92.353
(7) Epoch 010: | Train Loss: 0.24001 | Train Acc: 92.494
(7) Epoch 011: | Train Loss: 0.23616 | Train Acc: 92.622
(7) Epoch 012: | Train Loss: 0.23262 | Train Acc: 92.724
(7) Epoch 013: | Train Loss: 0.22964 | Train Acc: 92.814
(7) Epoch 014: | Train Loss: 0.22719 | Train Acc: 92.836
(7) Epoch 015: | Train Loss: 0.22451 | Train Acc: 92.909
(7) Epoch 016: | Train Loss: 0.22243 | Train Acc: 93.031
(7) Epoch 017: | Train Loss: 0.22039 | Train Acc: 93.018
(7) Epoch 018: | Train Loss: 0.

  0%|          | 0/50 [00:00<?, ?it/s]

(8) Epoch 001: | Train Loss: 0.93030 | Train Acc: 75.980
(8) Epoch 002: | Train Loss: 0.39772 | Train Acc: 88.845
(8) Epoch 003: | Train Loss: 0.32550 | Train Acc: 90.399
(8) Epoch 004: | Train Loss: 0.29349 | Train Acc: 91.134
(8) Epoch 005: | Train Loss: 0.27554 | Train Acc: 91.477
(8) Epoch 006: | Train Loss: 0.26390 | Train Acc: 91.729
(8) Epoch 007: | Train Loss: 0.25543 | Train Acc: 91.999
(8) Epoch 008: | Train Loss: 0.24881 | Train Acc: 92.180
(8) Epoch 009: | Train Loss: 0.24332 | Train Acc: 92.320
(8) Epoch 010: | Train Loss: 0.23896 | Train Acc: 92.459
(8) Epoch 011: | Train Loss: 0.23487 | Train Acc: 92.583
(8) Epoch 012: | Train Loss: 0.23157 | Train Acc: 92.639
(8) Epoch 013: | Train Loss: 0.22819 | Train Acc: 92.748
(8) Epoch 014: | Train Loss: 0.22566 | Train Acc: 92.816
(8) Epoch 015: | Train Loss: 0.22322 | Train Acc: 92.894
(8) Epoch 016: | Train Loss: 0.22099 | Train Acc: 92.969
(8) Epoch 017: | Train Loss: 0.21878 | Train Acc: 93.056
(8) Epoch 018: | Train Loss: 0.

  0%|          | 0/50 [00:00<?, ?it/s]

(9) Epoch 001: | Train Loss: 0.90151 | Train Acc: 75.837
(9) Epoch 002: | Train Loss: 0.39469 | Train Acc: 89.067
(9) Epoch 003: | Train Loss: 0.32431 | Train Acc: 90.499
(9) Epoch 004: | Train Loss: 0.29351 | Train Acc: 91.149
(9) Epoch 005: | Train Loss: 0.27627 | Train Acc: 91.593
(9) Epoch 006: | Train Loss: 0.26506 | Train Acc: 91.810
(9) Epoch 007: | Train Loss: 0.25694 | Train Acc: 92.024
(9) Epoch 008: | Train Loss: 0.25049 | Train Acc: 92.243
(9) Epoch 009: | Train Loss: 0.24500 | Train Acc: 92.386
(9) Epoch 010: | Train Loss: 0.24045 | Train Acc: 92.487
(9) Epoch 011: | Train Loss: 0.23655 | Train Acc: 92.590
(9) Epoch 012: | Train Loss: 0.23286 | Train Acc: 92.695
(9) Epoch 013: | Train Loss: 0.22980 | Train Acc: 92.763
(9) Epoch 014: | Train Loss: 0.22696 | Train Acc: 92.903
(9) Epoch 015: | Train Loss: 0.22443 | Train Acc: 92.889
(9) Epoch 016: | Train Loss: 0.22223 | Train Acc: 92.991
(9) Epoch 017: | Train Loss: 0.21997 | Train Acc: 93.082
(9) Epoch 018: | Train Loss: 0.

In [ ]:
import csv

data = list(zip(cv_test_accuracy, cv_test_precision, cv_test_f1, cv_test_roc))

csv_file_path = '/home/brunopsz/Data/acc_prec_f1_roc_10_cv_scanvi_final'

with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['accuracy', 'precision', 'f1', 'roc_auc']) 
    writer.writerows(data) 


In [18]:
print(precision_score(y_true_cells,y_pred_cells, average='weighted'))
print(f1_score(y_true_cells, y_pred_cells, average='weighted'))

0.9207599746649469
0.9205126109593935


In [19]:
accuracy_score(y_true_cells, y_pred_cells)

0.9209332469215813